# Regression

- [Linear Regression](#linreg)
- [Least Squares Algorithm and its Derivation](#LSA)
- [Categorical Variables and Linear Regression](#categlinreg)
- [Regression: Potential Problems](#problems)

<a id="linreg"></a>
# Linear Regression 

Simple Regression - just two varialbles, one which is the explanatory variable (x) and the other one is response variable (y). This type of regression can be easily illustrated with scatterplot. 

### Correlation Coefficent 

<b>Correlation Coefficient (r)</b> - the strength and direction of a linear relationship. $r \in [-1,1]$

The boundaries for the strengh of correlation depend on the fiels. General guidelines:  
- Strong: $0.7 \leq |r| < 1.0$   
- Moderate: $0.3 \leq |r| < 0.7$   
- Weak: $0.0 \leq |r| < 0.3$   


$$r = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y}) } { {\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2}} {\sqrt{\sum_{i=1}^n (y_i - \bar{y})^2}}} $$

<b>Important:</b> if $r=0$ it DOES NOT necessarily mean that there is no relationship at all. It just means that there is no <b>linear</b> relationships. So, correlation cofficient only captures linear relationships.

### Example in Python

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('quizzes-data-1.xlsx')
df = df[df.Temp.notnull()&df.Sales.notnull()]

In [4]:
np.corrcoef(df.Temp, df.Sales)

array([[1.        , 0.95902026],
       [0.95902026, 1.        ]])

### Regression Equation

$b_0$: The **intercept** is defined as the **predicted value of the response when the x-variable is zero**.

$b_i, i \geq 1$ : The **slope**: for every unit increase in x **the expected change(increase or decrease) in y by the slope, holding all else (other variables) constant**. 

The regression equation is as follows: 

$$\hat{y} = b_0 + b_1 x_1 + ... + b_n x_n$$

$b_0, b_1, ... ,b_n$ are statistic values, whereas $\beta_0, \beta_1,..., \beta_n$ are actual, population parameters. Also, $\hat{y}$ is predicted value, whereas $y$ is actual value.

<a id = "lsa"></a>
# Least Squares Algorithm  and its Derviation
Goal: Minimize the sum of the squared vertical distances from the line to points. Objective function will look like this:  

$$E = \sum_{i=1}^n (y_i-\hat{y_i})^2$$

Other loss functions are possible, but this one is the easiest one to work with since it's easy to take it's derivative which is necessary for finding the minimum. 

#### Derivation

We can define our $\hat{y}^{(i)}$ as:
$$(x^{(i)})^T b$$, 
where $x^{(i)}$ is a vector $[1; {x_{o}^{(i)}}]$ (I'm turning original scalar $x_{o}^{(i)}$ to a vector so that we can pack it into a dot product and then to matrix-vector multiplication. And for the sake of generalisation let's actually make $x^{(i)}$ n-dimensional. I.e. our i-th observation has n features.   
By default, all vectors are column vectors. Now, b is a vector $(b_0, b_1, ... b_n)$. Check that we get the same result after these arrangements: 

$$(x^{(i)})^T b = [1; x_1,...,x_n]^T [b_0, b_1,.. b_n] = b_0 + b_1 x^{(i)}_1 + b_2 x^{(i)}_2 + ... + b_n x^{(i)}_n $$

So we can rewrite our objective function as: 

$$E(b) = \sum_{i=1}^{n} (y^{(i)} - (x^{(i)})^T b)^2$$ 

This sum is actually the definiton of a dot product, so we can further rewrite it as: 

$$E(b) = \sum_{i=1}^{n} (y^{(i)} - (x^{(i)})^T b)^2 = (y-Xb)^T (y-Xb)$$, 

where X is a n by 2 matrix with the first column being all 1s. So when we multiply this matrix by vector b, we'll get $\hat{y}$ vector of predictions. Now we can minimize this function, but first we will expand it: 

$$E(b) = (y-Xb)^T (y-Xb) = y^T y - y^T X b - b^T X^T y + b^T X^T X b$$

Here, it's important to notice that $y^T X b = b^T X^T y$, so we can now write: 

$$E(b) = y^T y - 2 b^T X^T y + b^T X^T X b $$

And now we will take the derivative of this guy and equate it to $\vec{0}$:  

$$\nabla{E} = - 2 X^T y + 2 X^T X b = \vec{0} $$

And now we can find the b vector as follows: 

$$X^T y = X^T X b$$

$$b = (X^T X)^{-1} X^T y $$  

The only possible problem here is that the matrix might appear to be non-invertible and in this case there are special techniques that help to avoid it. Typically, pseudoinverse is used. 

#### Example in Python

In [5]:
df = pd.read_csv('data/house_prices.csv')
# add ones column
df['intercept'] = 1 
X = df[['intercept', 'area', 'bathrooms', 'bedrooms']]
y = df['price']

In [6]:
b = np.dot(np.dot(np.linalg.pinv(np.dot(X.transpose(),X)), X.transpose()),y)

In [7]:
b

array([10072.10704941,   345.91101884,  7345.39171708, -2925.80632748])

Or, using libraries: 

In [8]:
import statsmodels.api as ss

In [9]:
lm = ss.OLS(df['price'], df[['intercept', 'area', 'bathrooms', 'bedrooms']])
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                     4230.
Date:                Thu, 13 Dec 2018   Prob (F-statistic):               0.00
Time:                        19:11:56   Log-Likelihood:                -84517.
No. Observations:                6028   AIC:                         1.690e+05
Df Residuals:                    6024   BIC:                         1.691e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   1.007e+04   1.04e+04      0.972      0.331   -1.02e+04    3.04e+04
area         345.9110      7.227     47.863      0.000     331.743     360.079
bathrooms   7345.3917   1.43e+04      0.515      0.607   -2.06e+04    3.53e+04
bedrooms   -2925.8063   1.03e+04     -0.285      0.775    -2.3e+04    1.72e+04
==============================================================================
Omnibus:                      367.658   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              350.116
Skew:                           0.536   Prob(JB):                     9.40e-77
Kurtosis:                       2.503   Cond. No.                     1.16e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.16e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Results interpretation

#### p-values
The p-values are the probabilites of $b_i$ to be 0. This actually shows us the "usefullness of these parameters. In this case we see that area is a good predictor, while others are not.  

**Significant bivariate relationships are not always significant in multiple linear regression**

#### R-squared

R-squared - the amount of variability in the response (y) explained by the model. Closer to 1 - better fit. In fact, R-squared is $r^2$.

<a id="categlinreg"></a>
# Categorical variables in Linear Regression

## 1/0 encoding and Dummy Variables Trap
One way to work with categorical variables is to encode them as dummy variables. I.e. create a column for each value of the categoric variable and then encode with 1 or 0 it's presence or absence in the original column. 

### Example

In [10]:
np.random.seed(2)
values = np.random.choice(['A','B','C'],  size=5)
pd.DataFrame(values)

,0
0,A
1,B
2,A
3,C
4,C


In [11]:
X = pd.get_dummies(values)
X

,A,B,C
0,1,0,0
1,0,1,0
2,1,0,0
3,0,0,1
4,0,0,1


It all looks well and good and we don't see any linear dependencies here. However, we always add the intercept before fitting the regression line, according to the derivation shown in the previous section. So let's look at it now: 

In [12]:
X['intercept'] = 1
X

,A,B,C,intercept
0,1,0,0,1
1,0,1,0,1
2,1,0,0,1
3,0,0,1,1
4,0,0,1,1


Now there is definitely a dependence, because A + B + C = intercept. Which means we will have problems inverting $X^T X$. The problem is that X^T and X will contain linearly dependent columns, too: 

In [13]:
XtX = np.dot(X.transpose(),X)
XtX

array([[2, 0, 0, 2],
       [0, 1, 0, 1],
       [0, 0, 2, 2],
       [2, 1, 2, 5]])

We clearly see that column 4 is actually the sum of columns 1-3. So this is basically a **singular matrix** and we can't invert it, using traditional methods. 

In [14]:
try:
    np.linalg.inv(XtX)
except Exception as e:
    print(e)

Singular matrix


The pseudoinvers (np.linalg.pinv) will still work, but it's just logically incorrect in this case. 

### The baseline

When we drop one column and then run regression, we actually then use this column as a baseline. And the coefficient for the remaining dummy columns allow us to compare with the baseline. Below is the full example of running linear regression with categorical variables. Note that for the column 'neighborhood' we have 3 values: A,B,C, but we use only B and C. A is our baseline. The same for the style: ranch is our baseline, so we only use lodge and victorian.

In [15]:
df = pd.read_csv('data/house_prices.csv')
df_new = df.join(pd.get_dummies(df.neighborhood))
df_new = df_new.join(pd.get_dummies(df['style']))
df_new['intercept'] = 1
lm = ss.OLS(df_new['price'], df_new[['intercept','B','C', 'lodge','victorian', 'bathrooms','bedrooms']])
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.809
Method:                 Least Squares   F-statistic:                     4250.
Date:                Thu, 13 Dec 2018   Prob (F-statistic):               0.00
Time:                        19:12:04   Log-Likelihood:                -82944.
No. Observations:                6028   AIC:                         1.659e+05
Df Residuals:                    6021   BIC:                         1.659e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept  -3.833e+05    1.2e+04    -31.995      0.000   -4.07e+05    -3.6e+05
B           5.229e+05   7040.928     74.271      0.000    5.09e+05    5.37e+05
C          -7168.6285   7639.254     -0.938      0.348   -2.21e+04    7807.045
lodge       1.685e+05   9906.629     17.012      0.000    1.49e+05    1.88e+05
victorian   7.056e+04   8337.790      8.463      0.000    5.42e+04    8.69e+04
bathrooms   9.996e+04   1.09e+04      9.164      0.000    7.86e+04    1.21e+05
bedrooms    1.732e+05   7677.152     22.558      0.000    1.58e+05    1.88e+05
==============================================================================
Omnibus:                      978.611   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2926.472
Skew:                           0.848   Prob(JB):                         0.00
Kurtosis:                       5.962   Cond. No.                         25.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Here is how we can interpret the baseline. B is more expensive than A, because it has positive coefficient and actually adds about 5e5 to the price. C is, however, cheaper than A because of negative coefficient. The same goes for the styles. Both victorian and lodge styles are more expensive than ranch. 
Based on the summary, we can say that:
- there is statistically significant evidence that the average home price in B differs from the average home price in A (because the p-value is small). 
- there is no statistically significant evidence that the average home price in C differs from the average home price in A, because the p-value is big 
- there is statistically significant evidence that the average home price in B differs from the average home price in C because the confidence intervals for them don't overlap.  

### Alternative -1/0/1 Encoding

Another way to ecode categorical variables is with -1/0/1 encoding. This way in the resulting model we will be comparing not to the baseline but to the average, regardless of what column we dropped. In Python there is no automated way to do that - we have to write our own function.

<a id = "problems"></a>
# Regression: Potential Problems 

## 1. Non-linearity of the response-predictor relationships
Sometimes the relationships between the variables are far from being linear. In this case the whole model will be useless. We can check for linearity by making a plot of residuals $y-\hat{y}$ vs x or, in the case of multiple regression, vs $\hat{y}$. **Ideally**, the plot should show **NO** pattern. 

If there are curvature patterns in this plot, it suggests that a linear model might not actually fit the data, and some other relationship exists between the predictor variables and response.  

Examples of residuals plot: 

<p><img src="residuals.jpg" width="600" align='center'></p>
<i>Source: Udacity</i>

If we do identify non-linear relationships, then the simplest approach is to perform transformations of the features - for example, we can use $\log x$, $\sqrt{x}$ or $x^2$ instead of x. 

## 2. Correlation of error terms

When we're using linear regression, we're actually making one important assumption that the errors $\epsilon_1, ... \epsilon_n$ are uncorrelated (we can't predict $\epsilon_{n+1}$ with $\epsilon_n$. If the errors are correlated, then we get an inadequate picture of the goodness of our model: we get too narrow confidence intervals, too low p-values, etc. The main problem is that we will better predict adjacent event, but the model will be still not as good as it might seem. 

Correlated error are often the case in time series when the observation are taken at adjacent time points. To check for that, we shoul again **plot the residuals** as a function of time. If the adjacent errors are close to each other, then the errors are correlated. Also, there is a Durbin-Watson test that checks for correlation of error terms at lag 1. 

ARIMA and ARMA models actually use autocorrelaction for predictions.

## 3. Non-constant Variance and Normally Distributed Errors
Another assumption we make when creating a regression model is that the variance is constant - $\text{Var}({\epsilon_i})=\sigma^2$. To check for that, we again **plot the residuals**. A sign of non-constant variance is the funnel-shaped plot (plot d.). The problem here is that larger response produces larger error.  Non-constant variace leads to confidence intervals and p-values that are inaccurate.

One of the solutions to that is to transform the response variable, Y, by using something like $\log (Y)$ or $\sqrt{Y}$. Often, the <a href="https://www.statisticshowto.datasciencecentral.com/box-cox-transformation/">Box-Cox transformation</a> is used.

## 4. Outliers/ High leverage points
Generally speaking, outlier is a response that lies far away from the regular trends. They can appear as a result of a typo or incorrect recording of information. 
In order to check for the outliers we again plot the residuals. However, a better way is to plot *studentized* residuals - each residual divided by it's estimated standard error. 
If studentized value is greater than 3, then it's considered tobe an outlier. 

**High leverage point** - a data point of predictor featurs that is significantly different from other points. It **significantly** affects the the results and we can get much better fit if we get rid of such points. It can be hard to identify such point in multiple regression case: the features can be normal on their own, but be abnormal when combined. For simle linear regression we can identify high leverage point using the following formula for each point: 

$$h_i = \frac{1}{n}+ \frac{(x_i - \bar{x})^2}{\sum_{i'=1}^n (x_i'-\bar{x})^2}$$

High values of $h_i$ indicate high leverage points. 

## 5. Collinearity

Collinear features are features that are closely related to each other. It can cause problems because it will be hard to distinguish the effects of these predictors separately - as a result, we have multiple possible combinations of features for similar values of loss function, while when the featurs are not collinear, the minimum is unique and well defined. With collinear features we have a lot of uncertainty because of that variety of combination for similar values of minimum. Other consequences: 
- reduces the accuracy of coefficients esitmation 
- standard error fo $\beta_i$ increases 
- t-statistic (defined as $\frac{\beta_i}{SE_{\beta}}$) decreases  
- p-values increase 
- we fail to reject $H_0: \beta_i = 0$  
- we can even get flipped coefficients in the resulting model!!!

**Solution:** look at the correlation matrix of the predictors. High values indicate highly correlated pairs. Below is the correlation matrix for the predictors for the houses dataset, described in previous section. The matrix only shows the correlation coefficients bigger than 0.5. 

In [25]:
correlations = df_new[['area','bedrooms','bathrooms','A','B','C','lodge','ranch','victorian']].corr()
correlations[correlations>0.5]

,area,bedrooms,bathrooms,A,B,C,lodge,ranch,victorian
area,1.000000,0.901623,0.891481,NaN,NaN,NaN,NaN,NaN,0.678340
bedrooms,0.901623,1.000000,0.972768,NaN,NaN,NaN,NaN,NaN,0.705078
bathrooms,0.891481,0.972768,1.000000,NaN,NaN,NaN,NaN,NaN,0.692416
A,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
B,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
C,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
lodge,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
ranch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
victorian,0.678340,0.705078,0.692416,NaN,NaN,NaN,NaN,NaN,1.000000


Another problem - **multicollinearity**, when collinearity is between more than two variables, which is hard to identify since we can't get a correlation matrix that easily. In this case we calculate **variance inflation factor (VIF)**: 

$$\text{VIF}(\hat{\beta}) = \frac{1}{1-R^2_{X_j | X_{-j}}}$$
$R^2_{X_j | X_{-j}}$ is the regression of $X_j$ onto all other variables, excluding $X_j$. 

The minimum value for **VIF** is 1. When VIF = 1, it means there is no collinearity. Anything above 5 (or sometimes 10) signals about collinearity. More on VIF - [here](https://onlinecourses.science.psu.edu/stat501/node/347/)

Let's run an experiment on the data we have: 

In [184]:
features = ['intercept','area','bedrooms','bathrooms', 'B','C','lodge','victorian']
X = df_new[features]

In [192]:
def vif(df: pd.DataFrame, idx: int) -> float:
    predictors = df.columns
    this_predictor = predictors[idx]
    other_predictors = np.append(predictors[0:idx], predictors[idx+1:])
    lm = ss.OLS(df_new[this_predictor], df_new[other_predictors]).fit()
    return np.round(1/(1-lm.rsquared),4)

In [193]:
vifs = pd.DataFrame({'feature': features, 'VIF': [vif(X, i) for i in range(len(features))]})
vifs

,feature,VIF
0,intercept,17.4840
1,area,5.9843
2,bedrooms,22.3125
3,bathrooms,19.1940
4,B,1.3707
5,C,1.3708
6,lodge,1.9706
7,victorian,2.0465


There is also a method is statsmodels package that calculates VIF. It can be used as follows: 

In [187]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [188]:
vifs2 = pd.DataFrame({'feature': features,'VIF': [variance_inflation_factor(X.values, i) for i in range(len(features))]})
vifs2

,feature,VIF
0,intercept,17.483993
1,area,5.984297
2,bedrooms,22.312460
3,bathrooms,19.194018
4,B,1.370705
5,C,1.370759
6,lodge,1.970631
7,victorian,2.046512


As we can see, the ready algorithm works a bit differently, probably processing negative R-squared somehow. However, it doesn't influence our interpretation. B,C, lodge and victorian are all "good" features, while area, bedrooms and bathroms have high inflation factor and it is reasonable to get rid at least from one of them. Let's see what happens if we remove one variable: 

In [189]:
features2 = ['intercept', 'area','bathrooms', 'B','C','lodge','victorian']
X2 = df_new[features2]
vifs3 = pd.DataFrame({'feature': features2,'VIF': [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]})
vifs3

,feature,VIF
0,intercept,12.023573
1,area,5.279540
2,bathrooms,6.715363
3,B,1.370560
4,C,1.370757
5,lodge,1.877402
6,victorian,2.020455


Now the values look reasonable. The value for area is still too big and looks correlated with victorian, but it's still under 5, so we can live with it. 

**Another solution to multicollinearity** is to actuall combine the values somehow. 